In [4]:
from pathlib import Path

import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.models import Legend, LegendItem
from bokeh.palettes import RdYlBu4 as palette

import aframe_plots.plot_utils as utils

output_notebook()

Loading BokehJS ...

In [5]:
weeks_dates_map = {
    '01-18-2020_01-25-2020': 32,
    '06-15-2019_06-22-2019': 1,
    '06-22-2019_06-29-2019': 2,
    '07-06-2019_07-13-2019': 4,
    '08-31-2019_09-07-2019': 12,
    '11-23-2019_11-30-2019': 24,
}

data_dir = Path("data/interval-sd-vs-far")
dfs = {}
for data_file in data_dir.iterdir():
    week = weeks_dates_map[data_file.stem]
    dfs[week] = pd.read_csv(data_file, index_col=0)

In [6]:
fars = np.array([0.5, 1, 2, 5])
sd_interped = {}

for week, df in dfs.items():
    far = df["x"]
    sd = df["Log Normal 35/35"]
    sd_interped[week] = np.interp(fars, far, sd)

sd_interped_transpose = {}
for i, far in enumerate(fars):
    sd_interped_transpose[far] = [sd[i] for sd in sd_interped.values()]

In [7]:
p = utils.get_figure(
    title="Sensitive Distance Over Time for 35/35 Lognormal",
    x_axis_label="Weeks After Original Test Period",
    y_axis_label="Sensitive Distance (Mpc)",
    )

legends = []
for i, far in enumerate(fars):
    c = p.circle(
        x=list(dfs.keys()),
        y=sd_interped_transpose[far],
        size=10,
        color=palette[i]
    )

    legends.append(LegendItem(renderers=[c], label=f"FAR {far}/month"))

legend = Legend(items=legends, click_policy="mute")
p.add_layout(legend, "right")

show(p)